In [0]:
%run "../helpers/configuration"

In [0]:
races_df = spark.read.parquet(f"{processed_folder_path}/races").drop('ingestion_date', 'data_source')

In [0]:
circuits_df = spark.read.parquet(f"{processed_folder_path}/circuits").drop('ingestion_date', 'data_source')

In [0]:
drivers_df = spark.read.parquet(f"{processed_folder_path}/drivers").drop('ingestion_date', 'data_source')

In [0]:
constructors_df = spark.read.parquet(f"{processed_folder_path}/constructors").drop('ingestion_date', 'data_source')

In [0]:
results_df = spark.read.parquet(f"{processed_folder_path}/results").drop('ingestion_date', 'data_source')

In [0]:
races_circuits_df = races_df.join(circuits_df, "circuit_id", "left") \
    .drop("round", "circuit_ref", circuits_df.name, "country", "latitude", "longitude", "altitude", "circuit_id") \
    .withColumnRenamed("name", "race_name") \
    .withColumnRenamed("race_timestamp", "race_date") \
    .withColumnRenamed("location", "circuit_location")

In [0]:
results_races_circuits_df = results_df.join(races_circuits_df, "race_id", "left") \
    .drop("number", "position", "position_text", "position_order", "laps", "milliseconds", "rank", "fastest_lap_speed", "fastest_lap_time") \
    .withColumnRenamed("time", "race_time")

In [0]:
results_drivers_df = results_races_circuits_df.join(drivers_df, "driver_id", "left") \
    .drop("driver_ref", "code", "dob") \
    .withColumnRenamed("name", "driver_name") \
    .withColumnRenamed("number", "driver_number") \
    .withColumnRenamed("nationality", "driver_nationality")

In [0]:
results_constructors_df = results_drivers_df.join(constructors_df, "constructor_id", "left") \
    .drop("constructor_ref", "nationality") \
    .withColumnRenamed("name", "team")

In [0]:
from pyspark.sql.functions import current_timestamp
final_df = results_constructors_df.withColumn("created_date", current_timestamp()) \
    .drop("constructor_id", "driver_id", "race_id", "result_id")

In [0]:
%run "../helpers/functions"

In [0]:
write_parquet(final_df, f"{presentation_folder_path}/race_results")

In [0]:
display(spark.read.parquet(f"{presentation_folder_path}/race_results"))